# I. $LU$ factorization of a square matrix
When we premultply $A$ by lower triangular elemetary matrices $\Lambda_j$ it is trasfrommed in an  upper triangular matrix $U$

$$  \Lambda_{n-1}  \ldots\Lambda_2  \Lambda_1 A  = U$$

$$  A  = (\Lambda_{n-1}  \ldots\Lambda_2  \Lambda_2 )^{-1} U$$

The inverse of a product of matrices is the revesre product of inverses
$$  A  = (\Lambda_1^{-1}  \Lambda_2^{-1} \ldots  \Lambda_{n-1}^{-1}) U$$

and inverse of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_j^{-1} = - \Lambda_j$, so 

$$ A  =  (-\Lambda_1) (-\Lambda_2) \ldots   (-\Lambda_{n-1})  U$$

So the $LU$ column pivot factorization is 
$$  A  = L U$$
with
$$ U = \Lambda_{n-1}  \ldots\Lambda_2  \Lambda_1 A  $$
an upper triangular matrix
$$ L  =  \Lambda_1^{-1}  \Lambda_2^{-1} \ldots  \Lambda_{n-1}^{-1} $$
an lower triangular matrix.

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [1]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
        print("L \n",np.array(L))
        print("U \n",np.array(u))
        print("lam \n",np.array(lam))
    return L, u

In [2]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [3]:
a

array([[3.        , 3.        , 3.        , 3.        , 3.        ,
        3.        ],
       [3.        , 1.875     , 1.36363636, 1.07142857, 0.88235294,
        0.75      ],
       [3.        , 1.36363636, 0.88235294, 0.65217391, 0.51724138,
        0.42857143],
       [3.        , 1.07142857, 0.65217391, 0.46875   , 0.36585366,
        0.3       ],
       [3.        , 0.88235294, 0.51724138, 0.36585366, 0.28301887,
        0.23076923],
       [3.        , 0.75      , 0.42857143, 0.3       , 0.23076923,
        0.1875    ]])

In [4]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [5]:
L, u = diy_lu(a)

print('L\n', L, "\n")
print('U\n', u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

L 
 [[1. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 1.]]
U 
 [[ 3.     3.     3.     3.     3.     3.   ]
 [ 0.    -1.125 -1.636 -1.929 -2.118 -2.25 ]
 [ 0.    -1.636 -2.118 -2.348 -2.483 -2.571]
 [ 0.    -1.929 -2.348 -2.531 -2.634 -2.7  ]
 [ 0.    -2.118 -2.483 -2.634 -2.717 -2.769]
 [ 0.    -2.25  -2.571 -2.7   -2.769 -2.812]]
lam 
 [[1. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 1.]]
L 
 [[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 0.    1.    0.    0.   ]
 [1.    1.882 0.    0.    1.    0.   ]
 [1.    2.    0.    0.    0.    1.   ]]
U 
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+0

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [6]:
a1 = a.copy()
a1[1, 1] = 3

In [7]:
np.linalg.matrix_rank(a1)

6

In [8]:
l, u = diy_lu(a1)

print(l, u)

L 
 [[1. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 1.]]
U 
 [[ 3.     3.     3.     3.     3.     3.   ]
 [ 0.     0.    -1.636 -1.929 -2.118 -2.25 ]
 [ 0.    -1.636 -2.118 -2.348 -2.483 -2.571]
 [ 0.    -1.929 -2.348 -2.531 -2.634 -2.7  ]
 [ 0.    -2.118 -2.483 -2.634 -2.717 -2.769]
 [ 0.    -2.25  -2.571 -2.7   -2.769 -2.812]]
lam 
 [[1. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 1.]]
L 
 [[ 1. nan  0.  0.  0.  0.]
 [ 1. nan  0.  0.  0.  0.]
 [ 1. nan  1.  0.  0.  0.]
 [ 1. nan  0.  1.  0.  0.]
 [ 1. nan  0.  0.  1.  0.]
 [ 1. nan  0.  0.  0.  1.]]
U 
 [[ 3.     3.     3.     3.     3.     3.   ]
 [ 0.     0.    -1.636 -1.929 -2.118 -2.25 ]
 [   nan    nan   -inf   -inf   -inf   -inf]
 [   nan    nan   -inf   -inf   -inf   -inf]
 [   nan    nan   -inf   -inf   -inf   -inf]
 [   nan    nan   -inf   -inf   -inf   -inf]]
lam 
 [[  1.   0.  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in matmul
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [9]:
def leading_minors_test(A):
   """
    Function to check all leading Minors of a Matrix not equal to 0
   """
   # ... ENTER YOUR CODE HERE ...
   #Si algun Menor principal es cero retornara Falso
   for i in range(0, len(A)):
     if (np.linalg.det(A[0:i, 0:i]) == 0):
       return False
   return True

leading_minors_test(a), leading_minors_test(a1)

(True, False)

### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

# 2. $LU$ factorization column pivoting
When we premultply $A$ by elementary permutation matricex$P_j$( to find a good pivot) and the premultply by lower triangular elemetary matrices $\Lambda_j$  it is trasfrommed in an  upper triangular matrix $U$

$$  \Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1 A  = U$$
$$  A  = (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1} U$$

with

\begin{array}{ll}L  &=  (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1}\\
&= P_1^{-1} \Lambda_1^{-1} P_2^{-1} \Lambda_2^{-1} \ldots  P_{n-1}^{-1}\\
 &= P_1^{t} (-\Lambda_1) P_2^{t} (-\Lambda_2) \ldots  P_{n-1}^{t} (-\Lambda_{n-1})
\end{array}



Because the inverse a of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_i^{-1} = - \Lambda_i$ and the inverse of a Permutation Matrix (in particular an elementary permutation matrix)  is its transpose [math.stackexchange](
        https://math.stackexchange.com/questions/98549/the-transpose-of-a-permutation-matrix-is-its-inverse#:~:text=Taking%20the%20transpose%20of%20P,Pt%3DP%E2%88%921.)  $P_i^{-1}=P_i^{t}$, so


If we use 
$$  \hat{L}  = (\Lambda_{n-1}  \ldots \Lambda_2\Lambda_1)^{-1}$$
The inverse of a product of matrices is the revesre product of inverses 
$$ \hat{L} = \Lambda_1^{-1}\Lambda_2^{-1} \ldots  \Lambda_{n-1}^{-1} $$
the inverse a of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_i^{-1} = - \Lambda_i$
$$ \hat{L} = (-\Lambda_1)  (-\Lambda_2) \ldots  (-\Lambda_{n-1}) $$

Then
$$    A_\pi  = \hat{L} U$$
with $A_\pi$ equal to $A$ with rows permutated acoording to some permutation $\pi$

Equiavlently 
$$   A  = (\hat{L}U)_{\pi'}$$
 $(\hat{L}U)_{\pi'}$ equal to $\hat{L}U$ with rows permutated acoording to some permutation $\pi'$

In [ ]:
"""Construct the LU decomposition of the input matrix.
    
    LU decomposition with pivot: work column by column, accumulate elementary triangular matrices L @ np.transpose(Pj) .
    """

'Construct the LU decomposition of the input matrix.\n    \n    LU decomposition with pivot: work column by column, accumulate elementary triangular matrices L @ np.transpose(Pj) .\n    '

In [16]:
def diy_lu_column_pivot(a):

    N = a.shape[0]   

    u = a.copy()
    L = np.eye(N)
    P = np.eye(N)

    for j in range(N-1):
      row = np.argmax(np.abs(u[j:, j])) + j
      u[[j, row]] = u[[row, j]]

      Lj = np.eye(N)
      gamma = u[j+1:, j] / u[j, j]
      Lj[j+1:, j] = - gamma
      u = Lj @ u

      Lj[j+1:, j] = gamma

      L = L @ Lj

      Pj = np.eye(N)
      Pj[[j, row]] = Pj[[row, j]]
      P = P @ np.transpose(Pj)

    return L, u, P

In [17]:
N = 6
##a = np.zeros((N, N), dtype=float)
##for i in range(N):
 ##   for j in range(N):
  ##      a[i, j] = 3. / (0.6*i*j + 1)
##
a = [[3, 6, 8, 5, 8, 0],
[1, 2, 9, 5, 9, 7],
[4, 9, 7, 4, 1, 8],
[1, 6, 2, 7, 2, 1],
[0, 0, 4, 1, 3, 2],
[4, 0, 2, 2, 7, 7]]
a = np.array(a)
L, u, P = diy_lu_column_pivot(a)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n",L@u, "\n")
print("a\n",a, "\n")
print("P\n", P, "\n")
print("P@L@u\n", P@L@u, "\n")
print("a\n",a, "\n")

L
 [[ 1.     0.     0.     0.     0.     0.   ]
 [ 0.25   1.     0.     0.     0.     0.   ]
 [ 0.75   0.083  1.     0.     0.     0.   ]
 [ 0.25  -0.417 -0.248  1.     0.     0.   ]
 [ 0.     0.     0.541 -0.194  1.     0.   ]
 [ 1.     0.028  0.429  0.069 -0.156  1.   ]] 

u
 [[ 4.000e+00  9.000e+00  7.000e+00  4.000e+00  1.000e+00  8.000e+00]
 [ 0.000e+00 -9.000e+00 -5.000e+00 -2.000e+00  6.000e+00 -1.000e+00]
 [ 0.000e+00  0.000e+00  7.389e+00  4.056e+00  8.583e+00  5.028e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  6.173e+00  6.380e+00 -1.692e-01]
 [ 0.000e+00  0.000e+00  0.000e+00  5.551e-17  2.629e+00 -8.060e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  8.682e-18  0.000e+00 -2.015e+00]] 

L@u
 [[ 4.     9.     7.     4.     1.     8.   ]
 [ 1.    -6.75  -3.25  -1.     6.25   1.   ]
 [ 3.     6.    12.222  6.889  9.833 10.944]
 [ 1.     6.     2.     7.     2.     1.   ]
 [ 0.     0.     4.     1.     6.04  -5.305]
 [ 4.     8.75  10.028  6.111  4.877  9.361]] 

a
 [[3 6 8 5 8 0]
 [1 2 9 

In [18]:
a1 = a.copy()
a1[1, 1] = 3

L, u , P = diy_lu_column_pivot(a1)


print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a\n",a, "\n")
print("P\n", P, "\n")
print("P@L@u\n", P@L@u, "\n")
print("a\n",a, "\n")

L
 [[ 1.     0.     0.     0.     0.     0.   ]
 [ 0.25   1.     0.     0.     0.     0.   ]
 [ 0.75   0.083  1.     0.     0.     0.   ]
 [ 0.25  -0.417 -0.268  1.     0.     0.   ]
 [ 0.     0.     0.585 -0.201  1.     0.   ]
 [ 1.    -0.083  0.463  0.063 -0.521  1.   ]] 

u
 [[ 4.000e+00  9.000e+00  7.000e+00  4.000e+00  1.000e+00  8.000e+00]
 [ 0.000e+00 -9.000e+00 -5.000e+00 -2.000e+00  6.000e+00 -1.000e+00]
 [ 0.000e+00  0.000e+00  6.833e+00  3.833e+00  9.250e+00  4.917e+00]
 [ 0.000e+00  0.000e+00 -2.220e-16  6.195e+00  6.732e+00 -9.756e-02]
 [ 0.000e+00  0.000e+00  1.399e-17  0.000e+00  2.039e+00 -8.189e+00]
 [ 0.000e+00  0.000e+00 -3.729e-17  0.000e+00  0.000e+00 -5.166e+00]] 

L@u
 [[ 4.     9.     7.     4.     1.     8.   ]
 [ 1.    -6.75  -3.25  -1.     6.25   1.   ]
 [ 3.     6.    11.667  6.667 10.5   10.833]
 [ 1.     6.     2.     7.     2.     1.   ]
 [ 0.     0.     4.     1.     6.102 -5.291]
 [ 4.     9.75  10.583  6.333  4.148  9.458]] 

a
 [[3 6 8 5 8 0]
 [1 2 9 

In [19]:
a2 = np.array([[4,3,1], [5,7,0], [9,9,3], [8,2,4]])

L, u, P = diy_lu_column_pivot(a2)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a\n",a, "\n")
print("P\n", P, "\n")
print("P@L@u\n", P@L@u, "\n")
print("a\n",a, "\n")

L
 [[ 1.     0.     0.     0.   ]
 [ 0.556  1.     0.     0.   ]
 [ 0.444  0.167  1.     0.   ]
 [ 0.889 -0.333  0.455  1.   ]] 

u
 [[ 9.     9.     3.   ]
 [ 0.    -6.     1.333]
 [ 0.     0.    -1.222]
 [ 0.     0.     0.   ]] 

L@u
 [[ 9.     9.     3.   ]
 [ 5.    -1.     3.   ]
 [ 4.     3.     0.333]
 [ 8.    10.     1.667]] 

a
 [[3 6 8 5 8 0]
 [1 2 9 5 9 7]
 [4 9 7 4 1 8]
 [1 6 2 7 2 1]
 [0 0 4 1 3 2]
 [4 0 2 2 7 7]] 

P
 [[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]] 

P@L@u
 [[ 8.    10.     1.667]
 [ 4.     3.     0.333]
 [ 9.     9.     3.   ]
 [ 5.    -1.     3.   ]] 

a
 [[3 6 8 5 8 0]
 [1 2 9 5 9 7]
 [4 9 7 4 1 8]
 [1 6 2 7 2 1]
 [0 0 4 1 3 2]
 [4 0 2 2 7 7]] 



# 3. $LU$ factorization column pivoting and reconstruction
When we premultply $A$ by elementary permutation matricex$P_j$( to find a good pivot) and the premultply by lower triangular elemetary matrices $\Lambda_j$  it is trasfrommed in an  upper triangular matrix $U$

$$  \Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1 A  = U$$
$$  A  = (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1} U$$
$$  A  = (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1} U$$

The inverse of a product of matrices is the revesre product of inverses 
$$  A  = ( P_1^{-1} \Lambda_1^{-1} P_2^{-1} \Lambda_2^{-1} \ldots  P_{n-1}^{-1}\Lambda_{n-1}^{-1} ) U$$

So the $LU$ column pivot factorization is 
$$  A  = L U$$
with
$$ U = \Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1 A  $$



$$ L  = P_1^{-1} \Lambda_1^{-1} P_2^{-1} \Lambda_2^{-1} \ldots  P_{n-1}^{-1}\Lambda_{n-1}^{-1} $$

the inverse a of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_i^{-1} = - \Lambda_i$ and the inverse of a Permutation Matrix (in particular an elementary permutation matrix)  is its transpose [math.stackexchange](
        https://math.stackexchange.com/questions/98549/the-transpose-of-a-permutation-matrix-is-its-inverse#:~:text=Taking%20the%20transpose%20of%20P,Pt%3DP%E2%88%921.)  $P_i^{-1}=P_i^{t}$, so

$$ L  =  P_1^{t} (-\Lambda_1) P_2^{t} (-\Lambda_2) \ldots  P_{n-1}^{t} (-\Lambda_{n-1}) $$


In [22]:
def diy_lu_column_pivot_reconstruct(a):

    N = a.shape[0]   

    u = a.copy()
    L = np.eye(N)
    P = np.eye(N)

    for j in range(N-1):
      row = np.argmax(np.abs(u[j:, j])) + j
      u[[j, row]] = u[[row, j]]

      Pj = np.eye(N)
      Pj[[j, row]] = Pj[[row, j]]

      Lj = np.eye(N)
      gamma = u[j+1:, j] / u[j, j]
      Lj[j+1:, j] = - gamma
      u = Lj @ u

      Lj[j+1:, j] = gamma

      L = L @ np.transpose(Pj) @ Lj


    return L, u

In [27]:
N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

L, u = diy_lu_column_pivot_reconstruct(a)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a\n",a, "\n")

L
 [[1.    0.    0.    0.    0.    0.   ]
 [1.    0.5   1.    0.    0.    0.   ]
 [1.    0.727 0.706 1.    0.    0.   ]
 [1.    0.857 0.41  0.835 1.    0.   ]
 [1.    0.941 0.178 0.426 0.789 1.   ]
 [1.    1.    0.    0.    0.    0.   ]] 

u
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -3.506e-01 -5.786e-01 -7.330e-01 -8.438e-01]
 [ 0.000e+00  0.000e+00  2.776e-17  2.421e-02  4.866e-02  6.961e-02]
 [ 0.000e+00  2.220e-16 -2.317e-17  0.000e+00 -6.462e-04 -1.516e-03]
 [ 0.000e+00 -1.751e-16  6.463e-18  0.000e+00  0.000e+00  6.730e-06]] 

L@u
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]] 

a
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.8

In [24]:
a1 = a.copy()
a1[1, 1] = 3

L, u = diy_lu_column_pivot_reconstruct(a1)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a2\n",a1, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - a1\n",L@u - a1, "\n")

L
 [[1.    0.    0.    0.    0.    0.   ]
 [1.    0.    1.    0.    0.    0.   ]
 [1.    0.727 0.151 1.    0.    0.   ]
 [1.    0.857 0.088 0.514 1.    0.   ]
 [1.    0.941 0.038 0.208 0.641 1.   ]
 [1.    1.    0.    0.    0.    0.   ]] 

u
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.776e-17 -9.247e-02 -1.485e-01 -1.856e-01]
 [ 0.000e+00  2.220e-16 -1.427e-17  0.000e+00  1.841e-03  3.821e-03]
 [ 0.000e+00 -1.424e-16  3.391e-18  0.000e+00  0.000e+00 -1.233e-05]] 

L@u
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]] 

a2
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.

In [25]:
a2 = np.array([[4,3,1], [5,7,0], [9,9,3], [8,2,4]])

L, u = diy_lu_column_pivot_reconstruct(a2)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a2\n",a2, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - a2\n",L@u - a2, "\n")

L
 [[ 0.444  0.167  0.455  1.   ]
 [ 0.556 -0.333  1.     0.   ]
 [ 1.     0.     0.     0.   ]
 [ 0.889  1.     0.     0.   ]] 

u
 [[ 9.     9.     3.   ]
 [ 0.    -6.     1.333]
 [ 0.     0.    -1.222]
 [ 0.     0.     0.   ]] 

L@u
 [[4. 3. 1.]
 [5. 7. 0.]
 [9. 9. 3.]
 [8. 2. 4.]] 

a2
 [[4 3 1]
 [5 7 0]
 [9 9 3]
 [8 2 4]] 

L@u - a2
 [[0.00e+00 0.00e+00 2.22e-16]
 [0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00]] 

